In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(True)
clear_output()
Game.add_relationship('Go', 'Fh', At_War=1)
for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.Seasons[0][1]['Override']['Go'] = (22, 'enemy', 'Fh')
    Game.take_domain_actions()
    Game.war_move()
    Game.clean_up()
    Game.clear_screen()

df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
print(set(df['Action']))
print()
print(set(df[df['Success?']==True]['Action']))
Game.War


0
Instructions for updating:
Use tf.cast instead.
MOVE TROOPS Go ['Dwarf Guards', 'Goblin Infantry'] ['Kal-Saitharak', 'Kal-Saitharak'] Awallaigh
Forgot to mark a capitalitiative  9
Forgot to mark a capital
Forgot to mark a capital
Forgot to mark a capitalitiative  9
Forgot to mark a capital
Forgot to mark a capital
Forgot to mark a capitalitiative  9
Forgot to mark a capital
Forgot to mark a capital


NameError: name 'offence' is not defined

In [ ]:
Game.War.drop_duplicates().iloc[1]['Notes']

In [ ]:
df[df['Action'].str.contains('move')]

In [ ]:
Game.Troops[Game.Troops['Provence']=='Awallaigh']